In [1]:
import xlwings as xw
import pandas as pd

from Options_Math_Helpers import *
from Options_Math_Algebra import *
from Options_Math_Black_Scholes import *

oma  = OptionsMathAlgebra()
ombs = OptionsMathBlackScholes()

In [2]:
# Open workbook
wb = xw.Book('deribit.xlsx')

# Select sheet by name
sheet = wb.sheets['combo_with_time']

# Read entire used range into a DataFrame
raw_df = sheet.used_range.options(pd.DataFrame, header=True, index=False).value

In [3]:
cols = ['kind', 
        'instrument_name', 
        'option_type', 
        'strike', 
        'expiration_times_nyc', 
        'years_to_expiry', 
        'underlying_price',
        'mark_price', 
        'best_bid_price', 
        'best_ask_price',
        'mark_iv', 
        'greeks_delta',
        'greeks_gamma',
        'greeks_vega',
        'greeks_theta',
        'greeks_rho'
       ]

df = raw_df.loc[:, [c for c in cols if c in raw_df.columns]]
df = df.loc[df['kind'] == 'option']
df = df.drop(columns=['kind'])

print('done')

done


In [4]:
opt_types   = df['option_type']
fwds  = df['underlying_price']
strikes = df['strike']
times = df['years_to_expiry'] #* 365/365.25

df['mark_dollar']     = df['underlying_price'] * df['mark_price']
df['best_bid_dollar'] = df['underlying_price'] * df['mark_price'] #df['best_bid_price']  
df['best_ask_dollar'] = df['underlying_price'] * df['mark_price'] #df['best_ask_price'] 

In [5]:
df['d0'] = ombs.bs_d0(fwd=fwds, strike=strikes, vol=df['mark_iv']/100, time=times)
df['d1'] = ombs.bs_d1(vol=df['mark_iv']/100, time=times, d0=df['d0'])
df['d2'] = ombs.bs_d2(vol=df['mark_iv']/100, time=times, d0=df['d0'])

df['mark_delta'] = ombs.bs_delta(opt_type=opt_types, fwd=fwds, strike=strikes, vol=df['mark_iv']/100, time=times, d1=df['d1'])
df['mark_gamma'] = ombs.bs_gamma(opt_type=opt_types, fwd=fwds, strike=strikes, vol=df['mark_iv']/100, time=times, d1=df['d1'])
df['mark_vega']  = ombs.bs_vega (opt_type=opt_types, fwd=fwds, strike=strikes, vol=df['mark_iv']/100, time=times, d1=df['d1'])
df['mark_theta'] = ombs.bs_theta(opt_type=opt_types, fwd=fwds, strike=strikes, vol=df['mark_iv']/100, time=times, d1=df['d1'], d2=df['d2'])
df['mark_rho']   = ombs.bs_rho  (opt_type=opt_types, fwd=fwds, strike=strikes, vol=df['mark_iv']/100, time=times, d2=df['d2'])

In [6]:
df['delta_diff'] = df['greeks_delta'] - df['mark_delta']
df['gamma_diff'] = df['greeks_gamma'] - df['mark_gamma']
df['vega_diff']  = df['greeks_vega']  - df['mark_vega']
df['theta_diff'] = df['greeks_theta'] - df['mark_theta']
df['rho_diff']   = df['greeks_rho']   - df['mark_rho']

In [7]:
sheet = wb.sheets('greeks')
sheet.range('a1').options(index = False, header=1).value = df